# Radiómica

In [1]:
import os
import radiomics
from radiomics import firstorder, shape2D, glcm, glrlm, glszm, ngtdm, gldm, imageoperations
import SimpleITK as sitk
import numpy as np
import cv2
from skimage.io import imread
import pandas as pd
import os
import pandas as pd
import numpy as np
import SimpleITK as sitk
import cv2
from skimage.io import imread
from radiomics import firstorder, shape2D, glcm, glszm, glrlm, ngtdm, gldm, imageoperations


In [2]:
base_folder = '/home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/'
output_folder = '/home/anna/TFM/Radiomica/Radiomica/Características/'
os.makedirs(output_folder, exist_ok=True)

In [ ]:
import os
import numpy as np
import pandas as pd
import SimpleITK as sitk
from skimage.io import imread
from radiomics import firstorder, shape2D, glcm, glszm, glrlm, ngtdm, gldm, imageoperations

base_folder = '/home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/'
output_folder = '/home/anna/TFM/Radiomica/Radiomica/Características/'
os.makedirs(output_folder, exist_ok=True)

dfs0, dfs, dfs2, dfs3, dfs4, dfs5, dfs6, dfs7 = ([] for _ in range(8))

def get_image_and_mask_rgb(subfolder, base_folder):
    subfolder_path = os.path.join(base_folder, subfolder)
    img_path = os.path.join(subfolder_path, 'imagen_original.png')
    mask_path = os.path.join(subfolder_path, 'mascara_segmentada.png')

    if not (os.path.exists(img_path) and os.path.exists(mask_path)):
        raise FileNotFoundError(f"Faltan archivos en {subfolder_path}")

    print(f"Cargando imagen y máscara de: {subfolder_path}")

    image = imread(img_path)  
    mask = imread(mask_path, as_gray=True)
    mask = (mask > 0).astype(np.uint8)

    R, G, B = image[:, :, 0], image[:, :, 1], image[:, :, 2]

    sitk_mask = sitk.GetImageFromArray(mask)
    sitk_R = sitk.GetImageFromArray(R.astype(np.uint8))
    sitk_G = sitk.GetImageFromArray(G.astype(np.uint8))
    sitk_B = sitk.GetImageFromArray(B.astype(np.uint8))

    return (sitk_R, sitk_G, sitk_B), sitk_mask, img_path, mask_path

def extract_features(image, mask, suffix):
    image = imageoperations.normalizeImage(image)

    feats = {}

    extractor1 = firstorder.RadiomicsFirstOrder(image, mask)
    extractor1.enableAllFeatures()
    feats.update({f"firstorder_{k}_{suffix}": v for k, v in extractor1.execute().items()})

    extractor2 = shape2D.RadiomicsShape2D(image, mask)
    extractor2.enableAllFeatures()
    feats.update({f"shape2D_{k}_{suffix}": v for k, v in extractor2.execute().items()})

    extractor3 = glcm.RadiomicsGLCM(image, mask)
    extractor3.enableAllFeatures()
    feats.update({f"glcm_{k}_{suffix}": v for k, v in extractor3.execute().items()})

    extractor4 = glszm.RadiomicsGLSZM(image, mask)
    extractor4.enableAllFeatures()
    feats.update({f"glszm_{k}_{suffix}": v for k, v in extractor4.execute().items()})

    extractor5 = glrlm.RadiomicsGLRLM(image, mask)
    extractor5.enableAllFeatures()
    feats.update({f"glrlm_{k}_{suffix}": v for k, v in extractor5.execute().items()})

    extractor6 = ngtdm.RadiomicsNGTDM(image, mask)
    extractor6.enableAllFeatures()
    feats.update({f"ngtdm_{k}_{suffix}": v for k, v in extractor6.execute().items()})

    extractor7 = gldm.RadiomicsGLDM(image, mask)
    extractor7.enableAllFeatures()
    feats.update({f"gldm_{k}_{suffix}": v for k, v in extractor7.execute().items()})

    print(f"Extraídas {len(feats)} características del canal {suffix}")
    return feats

def split_features(feats_dict, suffix):
    f1 = {k: v for k, v in feats_dict.items() if 'firstorder' in k}
    f2 = {k: v for k, v in feats_dict.items() if 'shape2D' in k}
    f3 = {k: v for k, v in feats_dict.items() if 'glcm' in k}
    f4 = {k: v for k, v in feats_dict.items() if 'glszm' in k}
    f5 = {k: v for k, v in feats_dict.items() if 'glrlm' in k}
    f6 = {k: v for k, v in feats_dict.items() if 'ngtdm' in k}
    f7 = {k: v for k, v in feats_dict.items() if 'gldm' in k}
    return f1, f2, f3, f4, f5, f6, f7

def dict_to_df(d): 
    return pd.DataFrame([d])

subfolders = [f for f in os.listdir(base_folder) if os.path.isdir(os.path.join(base_folder, f))]

for i, subfolder in enumerate(subfolders):
    print(f"\nProcesando {i}: {subfolder}")

    try:
        (img_R, img_G, img_B), mask, img_path, mask_path = get_image_and_mask_rgb(subfolder, base_folder)
    except Exception as e:
        print(f"[ERROR] {e}")
        continue

    if np.sum(sitk.GetArrayFromImage(mask)) == 0:
        print(f"[AVISO] Máscara vacía en {subfolder}, se omite.")
        continue

    feats_R = extract_features(img_R, mask, "R")
    feats_G = extract_features(img_G, mask, "G")
    feats_B = extract_features(img_B, mask, "B")

    # DataFrame con nombres
    data0 = pd.DataFrame([[subfolder, subfolder]], columns=['Image', 'Mask'])

    f1R, f2R, f3R, f4R, f5R, f6R, f7R = split_features(feats_R, "R")
    f1G, f2G, f3G, f4G, f5G, f6G, f7G = split_features(feats_G, "G")
    f1B, f2B, f3B, f4B, f5B, f6B, f7B = split_features(feats_B, "B")

    dfs0.append(data0)
    dfs.append(pd.concat([dict_to_df(f1R), dict_to_df(f1G), dict_to_df(f1B)], axis=1))
    dfs2.append(pd.concat([dict_to_df(f2R), dict_to_df(f2G), dict_to_df(f2B)], axis=1))
    dfs3.append(pd.concat([dict_to_df(f3R), dict_to_df(f3G), dict_to_df(f3B)], axis=1))
    dfs4.append(pd.concat([dict_to_df(f4R), dict_to_df(f4G), dict_to_df(f4B)], axis=1))
    dfs5.append(pd.concat([dict_to_df(f5R), dict_to_df(f5G), dict_to_df(f5B)], axis=1))
    dfs6.append(pd.concat([dict_to_df(f6R), dict_to_df(f6G), dict_to_df(f6B)], axis=1))
    dfs7.append(pd.concat([dict_to_df(f7R), dict_to_df(f7G), dict_to_df(f7B)], axis=1))

# Guardar CSVs finales usando output_folder
pd.concat(dfs0).to_csv(os.path.join(output_folder, "nombres.csv"), index=False)
pd.concat(dfs).to_csv(os.path.join(output_folder, "firstorder.csv"), index=False)
pd.concat(dfs2).to_csv(os.path.join(output_folder, "shape.csv"), index=False)
pd.concat(dfs3).to_csv(os.path.join(output_folder, "glcm.csv"), index=False)
pd.concat(dfs4).to_csv(os.path.join(output_folder, "glszm.csv"), index=False)
pd.concat(dfs5).to_csv(os.path.join(output_folder, "glrlm.csv"), index=False)
pd.concat(dfs6).to_csv(os.path.join(output_folder, "ngtdm.csv"), index=False)
pd.concat(dfs7).to_csv(os.path.join(output_folder, "gldm.csv"), index=False)

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated



Procesando 0: 815674_300481_28-10-2020_DSC_0048
Cargando imagen y máscara de: /home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/815674_300481_28-10-2020_DSC_0048
Extraídas 102 características del canal R
Extraídas 102 características del canal G
Extraídas 102 características del canal B

Procesando 1: 712761-300309-300481-02
Cargando imagen y máscara de: /home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/712761-300309-300481-02
Extraídas 102 características del canal R
Extraídas 102 características del canal G
Extraídas 102 características del canal B

Procesando 2: 30745-291008-300481-01
Cargando imagen y máscara de: /home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/30745-291008-300481-01
Extraídas 102 características del canal R
Extraídas 102 características del canal G
Extraídas 102 características del canal B

Procesando 3: 424155_300481_1-3-2022_P1011472
Cargando imagen y máscara de: /home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Extraídas 102 características del canal R
Extraídas 102 características del canal G
Extraídas 102 características del canal B

Procesando 4: 69760-260405-300481
Cargando imagen y máscara de: /home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/69760-260405-300481
Extraídas 102 características del canal R
Extraídas 102 características del canal G
Extraídas 102 características del canal B

Procesando 5: 385091_300481_7-1-2019_DSC_0014
Cargando imagen y máscara de: /home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/385091_300481_7-1-2019_DSC_0014
Extraídas 102 características del canal R
Extraídas 102 características del canal G
Extraídas 102 características del canal B

Procesando 6: 898961-040209-300481-02
Cargando imagen y máscara de: /home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/898961-040209-300481-02


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


...

Extraídas 102 características del canal G
Extraídas 102 características del canal B

Procesando 508: 233981_300481_21-5-2021_DSC_0773
Cargando imagen y máscara de: /home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/233981_300481_21-5-2021_DSC_0773
Extraídas 102 características del canal R
Extraídas 102 características del canal G
Extraídas 102 características del canal B

Procesando 509: 212980_300481_30-4-2018_DSC_0325
Cargando imagen y máscara de: /home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/212980_300481_30-4-2018_DSC_0325
Extraídas 102 características del canal R
Extraídas 102 características del canal G
Extraídas 102 características del canal B

Procesando 510: 1331146_300481_15-6-2020_DSC_0192
Cargando imagen y máscara de: /home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/1331146_300481_15-6-2020_DSC_0192
Extraídas 102 características del canal R
Extraídas 102 características del canal G
Extraídas 102 características del canal B

Procesando

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Extraídas 102 características del canal R
Extraídas 102 características del canal G
Extraídas 102 características del canal B

Procesando 512: 462909-071206-02-300481
Cargando imagen y máscara de: /home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/462909-071206-02-300481
Extraídas 102 características del canal R
Extraídas 102 características del canal G
Extraídas 102 características del canal B

Procesando 513: 20056-z2023-300481-28062023-2
Cargando imagen y máscara de: /home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/20056-z2023-300481-28062023-2
Extraídas 102 características del canal R
Extraídas 102 características del canal G
Extraídas 102 características del canal B

Procesando 514: 111402_300481_10-12-Tuesday_DSC_0727
Cargando imagen y máscara de: /home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/111402_300481_10-12-Tuesday_DSC_0727
Extraídas 102 características del canal R
Extraídas 102 características del canal G
Extraídas 102 características

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated


Extraídas 102 características del canal R
Extraídas 102 características del canal G
Extraídas 102 características del canal B

Procesando 516: 160918_060215_300481 (3)
Cargando imagen y máscara de: /home/anna/TFM/Radiomica/Segmentacion/Segmentacion_resultados/160918_060215_300481 (3)
Extraídas 102 características del canal R
Extraídas 102 características del canal G
Extraídas 102 características del canal B


In [ ]:
df_names = pd.concat(dfs0, ignore_index=True)

df_features = pd.concat([pd.concat(dfs, ignore_index=True),
                         pd.concat(dfs2, ignore_index=True),
                         pd.concat(dfs3, ignore_index=True),
                         pd.concat(dfs4, ignore_index=True),
                         pd.concat(dfs5, ignore_index=True),
                         pd.concat(dfs6, ignore_index=True),
                         pd.concat(dfs7, ignore_index=True)],
                        axis=1)

df_final = pd.concat([df_names, df_features], axis=1)

df_final.to_csv(os.path.join(output_folder, "caracteristicas_rgb.csv"), index=False)


In [ ]:
import csv

archivo = '/home/anna/TFM/Radiomica/Radiomica/Características/caracteristicas_rgb.csv'

with open(archivo, 'r') as f:
    reader = csv.reader(f)
    encabezado = next(reader)  
    total_columnas = len(encabezado)
    caracteristicas = total_columnas - 2  
    print(f"Total columnas: {total_columnas}")
    print(f"Características (excluyendo Image y Mask): {caracteristicas}")


Total columnas: 308
Características (excluyendo Image y Mask): 306
